# Blue Brain BioExplorer
![](../../bioexplorer_banner.png)

# High Glucose Scenario

## Connect to BioExplorer backend

In [ ]:
from bioexplorer import BioExplorer, RNASequence, Protein, \
    AssemblyProtein, Virus, Surfactant, Membrane, Cell, Sugars, \
    Volume, Vector2, Vector3, Quaternion
import nglview

hostname = 'localhost:5000'
resource_folder = '../../../tests/test_files/'

be = BioExplorer(hostname)
core = be.core_api()
version = be.version()
be.reset()
print('Version: ' + str(version))

export_to_cache = False

In [ ]:
'''Accelerate loading by not showing models as they are loaded'''
status = be.set_general_settings(
    logging_enabled=True,
    model_visibility_on_creation=False)

## Global parameters

In [ ]:
# Scene
scene_size = 800.0

# Proteins
protein_radius_multiplier = 1.0
protein_representation = be.REPRESENTATION_ATOMS_AND_STICKS
protein_load_hydrogen = False

# Glycans
add_glycans = True
glycan_radius_multiplier = 1.0
glycan_representation = be.REPRESENTATION_ATOMS_AND_STICKS

# Viruses
nb_protein_s = 62
nb_protein_e = 42
nb_protein_m = 50
add_rna = False

# Immune system
nb_glucoses = 360000
nb_lactoferrins = 50
nb_defensins = 100
nb_defensins_on_virus = 2

# Cell
cell_name = 'Cell'
cell_position = Vector3(4.5, -186.0, 7.0)
cell_height = scene_size / 10.0
cell_nb_receptors = 100
cell_nb_lipids = 1200000

## Resources

In [ ]:
cache_folder = resource_folder + 'caches/' + version + '/'
image_folder = resource_folder + 'images/'
pdb_folder = resource_folder + 'pdb/'
rna_folder = resource_folder + 'rna/'
obj_folder = resource_folder + 'obj/'
glycan_folder = pdb_folder + 'glycans/'
membrane_folder = pdb_folder + 'membrane/'

complex_paths = [glycan_folder + 'complex/33.pdb', glycan_folder + 'complex/34.pdb',
                 glycan_folder + 'complex/35.pdb',glycan_folder + 'complex/36.pdb']
high_mannose_paths = [glycan_folder + 'high-mannose/1.pdb', 
                      glycan_folder + 'high-mannose/2.pdb',
                      glycan_folder + 'high-mannose/3.pdb',
                      glycan_folder + 'high-mannose/4.pdb']
hybrid_paths = [glycan_folder + 'hybrid/24.pdb']
o_glycan_paths = [glycan_folder + 'o-glycan/12.pdb']

glucose_path = pdb_folder + 'glucose.pdb'
lactoferrin_path=pdb_folder + 'immune/1b0l.pdb'
defensin_path = pdb_folder + 'immune/1ijv.pdb'

surfactant_head_source = pdb_folder + 'surfactant/1pw9.pdb'
surfactant_branch_source = pdb_folder + 'surfactant/1k6f.pdb'

## Coronaviruses

In [ ]:
nglview.show_file(pdb_folder + 'sars-cov-2-v1.pdb')

In [ ]:
nglview.show_file(pdb_folder + '6vyb.pdb')

In [ ]:
nglview.show_file(pdb_folder + 'QHD43419a.pdb')

In [ ]:
nglview.show_file(pdb_folder + 'QHD43418a.pdb')

In [ ]:
be.add_coronavirus(
    name='Coronavirus 1', resource_folder=resource_folder,
    representation=protein_representation,
    position=Vector3(-337.3, -92.3, -99.2), add_glycans=add_glycans)
be.add_coronavirus(
    name='Coronavirus 2', resource_folder=resource_folder,
    representation=protein_representation,
    position=Vector3(-74.9, -97.1, 228.8), add_glycans=add_glycans)
be.add_coronavirus(
    name='Coronavirus 3', resource_folder=resource_folder,
    representation=protein_representation,
    position=Vector3(187.5, -110.4, 51.2), add_glycans=add_glycans)
be.add_coronavirus(
    name='Coronavirus 4', resource_folder=resource_folder,
    representation=protein_representation,
    position=Vector3(4.5, 100.0, 7.5),
    rotation=Quaternion(0.0, 0.0, 0.0, 1.0), 
    add_glycans=add_glycans)
be.add_coronavirus(
    name='Coronavirus 5', resource_folder=resource_folder,
    representation=protein_representation,
    position=Vector3(73.9, -117.1, -190.4), add_glycans=add_glycans)
be.add_coronavirus(
    name='Coronavirus 6', resource_folder=resource_folder,
    representation=protein_representation,
    position=Vector3(211.5, -104.9, 339.2), add_glycans=add_glycans)

In [ ]:
if export_to_cache:
    '''Show loaded models'''
    status = be.set_models_visibility(True)
    '''Set materials'''
    be.apply_default_color_scheme(shading_mode=be.SHADING_MODE_BASIC)
    '''Export to cache file'''
    cache_filename = cache_folder + 'high_glucose_viruses.bioexplorer'
    if add_rna:
        cache_filename = cache_folder + 'high_glucose_open_viruses.bioexplorer'
    print(cache_filename)
    be.export_to_file(cache_filename)

## Host cell

In [ ]:
def add_cell(
        name, size, height, nb_receptors,
        position=Vector3(), random_seed=1):
    
    ace2_receptor = Protein(
        sources=[pdb_folder + '6m18.pdb'], occurences=nb_receptors,
        position=Vector3(0.0, 6.0, 0.0))
    
    membrane = Membrane(
        sources=[
            membrane_folder + 'segA.pdb',
            membrane_folder + 'segB.pdb',
            membrane_folder + 'segC.pdb',
            membrane_folder + 'segD.pdb'
        ],
        occurences=cell_nb_lipids
    )

    cell = Cell(
        name=name, size=size, extra_parameters=[height],
        shape=be.ASSEMBLY_SHAPE_SINUSOIDAL, 
        membrane=membrane, receptor=ace2_receptor,
        random_position_seed=1, random_position_strength=0.025,
        random_rotation_seed=2, random_rotation_strength=2.0
    )
    
    be.add_cell(
        cell=cell, position=position,
        representation=protein_representation,
        random_seed=random_seed)

    if nb_receptors != 0 and add_glycans:
        be.add_multiple_glycans(
            representation=glycan_representation, assembly_name=name, 
            glycan_type=be.NAME_GLYCAN_COMPLEX,
            protein_name=be.NAME_RECEPTOR, paths=complex_paths, 
            indices=[53, 90, 103, 322, 432, 690])
        be.add_multiple_glycans(
            representation=glycan_representation, assembly_name=name,
            glycan_type=be.NAME_GLYCAN_HYBRID,
            protein_name=be.NAME_RECEPTOR, paths=hybrid_paths, 
            indices=[546])

        indices = [[155, Quaternion(0.707, 0.0, 0.707, 0.0)],
                   [730, Quaternion(0.707, 0.0, 0.707, 0.0)]]
        for index in indices:
            o_glycan_name = name + '_' + be.NAME_GLYCAN_O_GLYCAN + '_' + str(index[0])
            o_glycan = Sugars(
                assembly_name=name, name=o_glycan_name, source=o_glycan_paths[0],
                protein_name=name + '_' + be.NAME_RECEPTOR, representation=glycan_representation,
                chain_ids=[2, 4], site_indices=[index[0]], rotation=index[1])
            be.add_sugars(o_glycan)

In [ ]:
add_cell(
    name=cell_name, nb_receptors=cell_nb_receptors,
    size=scene_size * 2.0, height=cell_height,
    position=cell_position, random_seed=10)

In [ ]:
'''Show loaded models'''
status = be.set_models_visibility(True)

In [ ]:
if export_to_cache:
    be.apply_default_color_scheme(shading_mode=be.SHADING_MODE_BASIC)
    be.set_protein_color_scheme(
        cell_name, cell_name + '_' + be.NAME_RECEPTOR,
        be.COLOR_SCHEME_CHAINS, 'OrRd_r', 7)
    cache_filename = cache_folder + 'cell.bioexplorer'
    print(cache_filename)
    be.export_to_file(cache_filename)

## Immune system

### Surfactant A and D

In [ ]:
nglview.show_file(surfactant_head_source)

In [ ]:
def add_surfactant_d(name, position, random_seed):
    surfactant_d = Surfactant(
        name=name, surfactant_protein=be.SURFACTANT_PROTEIN_D, 
        head_source=surfactant_head_source,
        branch_source=surfactant_branch_source)
    be.add_surfactant(
        surfactant=surfactant_d, representation=protein_representation, 
        position=position, random_seed=random_seed)

add_surfactant_d(
    name='Surfactant-D 1',
    position=Vector3(74.0, 24.0, -45.0),
    random_seed=1)
add_surfactant_d(
    name='Surfactant-D 2',
    position=Vector3(104.0, 175.0, -89.0),
    random_seed=2)
add_surfactant_d(
    name='Surfactant-D 3',
    position=Vector3(-260.0, 50.0, 0.0),
    random_seed=6)

In [ ]:
def add_surfactant_a(name, position, random_seed):
    surfactant_a = Surfactant(
        name=name, surfactant_protein=be.SURFACTANT_PROTEIN_A, 
        head_source=surfactant_head_source,
        branch_source=surfactant_branch_source)
    be.add_surfactant(
        surfactant=surfactant_a, representation=protein_representation, 
        position=position, random_seed=random_seed)

add_surfactant_a(
    name='Surfactant-A 1', position=Vector3(-100.0, 150.0, 0.0), random_seed=2)

In [ ]:
# Add glucose to surfactants
def add_glucose_to_surfactant_head(name):
    for index in [321, 323]:
        glucose_name = name + '_' + be.NAME_GLUCOSE + '_' + str(index)
        glucose = Sugars(
            assembly_name=name, name=glucose_name, source=glucose_path,
            protein_name=name + '_' + be.NAME_SURFACTANT_HEAD, 
            representation=glycan_representation, site_indices=[index])
        be.add_sugars(glucose)
add_glucose_to_surfactant_head('Surfactant-D 1')
add_glucose_to_surfactant_head('Surfactant-D 2')
add_glucose_to_surfactant_head('Surfactant-D 3')
add_glucose_to_surfactant_head('Surfactant-A 1')

### Glucose

In [ ]:
nglview.show_file(glucose_path)

In [ ]:
glucose = Protein(
    sources=[glucose_path], load_non_polymer_chemicals=True, 
    occurences=nb_glucoses)
volume = Volume(
    name=be.NAME_GLUCOSE, size=scene_size, protein=glucose)
status = be.add_volume(
    volume=volume, representation=protein_representation,
    position=Vector3(0.0, scene_size / 2.0 - 200.0, 0.0))

### Lactoferrins

In [ ]:
nglview.show_file(lactoferrin_path)

In [ ]:
lactoferrin = Protein(
    sources=[lactoferrin_path], load_non_polymer_chemicals=True,
    occurences=nb_lactoferrins)
lactoferrins_volume = Volume(
    name=be.NAME_LACTOFERRIN, size=scene_size, protein=lactoferrin)
status = be.add_volume(
    volume=lactoferrins_volume, representation=protein_representation,
    position=Vector3(0.0, scene_size / 2.0 - 200.0, 0.0),
    random_seed=2)

### Defensins

In [ ]:
nglview.show_file(defensin_path)

In [ ]:
defensin = Protein(
    sources=[defensin_path], load_non_polymer_chemicals=True, 
    occurences=nb_defensins)
defensins_volume = Volume(
    name=be.NAME_DEFENSIN, size=scene_size, protein=defensin)
status = be.add_volume(
    volume=defensins_volume, representation=protein_representation,
    position=Vector3(0.0, scene_size / 2.0 - 200.0, 0.0),
    random_seed=3)

### Materials

In [ ]:
be.apply_default_color_scheme(
    shading_mode=be.SHADING_MODE_DIFFUSE,
    specular_exponent=50)

In [ ]:
for model in core.scene.models:
    model_id = model['id']
    model_name = model['name']
    if be.NAME_COLLAGEN in model_name:
        material_ids = list(be.get_material_ids(model_id)['ids'])
        nb_materials = len(material_ids)
        print(model_name)
        palette = list()
        emissions = list()
        for i in range(nb_materials):
            palette.append([1,1,1])
            emissions.append(0.1)
        be.set_materials(
            model_ids=[model_id], material_ids=material_ids,
            diffuse_colors=palette, specular_colors=palette,
            emissions=emissions
        )
core.scene.commit()

In [ ]:
if export_to_cache:
    '''Show loaded models'''
    status = be.set_models_visibility(True)
    '''Export to cache file'''
    cache_filename = cache_folder + 'high_glucose_immune.bioexplorer'
    print(cache_filename)
    be.export_to_file(cache_filename)

## Export full scene to cache file

In [ ]:
if export_to_cache:
    '''Show loaded models'''
    status = be.set_models_visibility(True)
    '''Export to cache file'''
    cache_filename = cache_folder + 'high_glucose_scenario.bioexplorer'
    print(cache_filename)
    be.export_to_file(cache_filename)

## Rendering settings

In [ ]:
status = core.set_renderer(
    background_color=[96 / 255, 125 / 255, 139 / 255],
    current='bio_explorer', head_light=False,
    samples_per_pixel=1, subsampling=1, max_accum_frames=128)
params = be.core_api().BioExplorerRendererParams()
params.exposure = 1.0
params.gi_samples = 1
params.gi_weight = 0.3
params.gi_distance = 500
params.shadows = 1.0
params.soft_shadows = 0.02
params.fog_start = 1000
params.fog_thickness = 300
params.max_bounces = 1
params.use_hardware_randomizer = True
status = core.set_renderer_params(params)

status = core.clear_lights()
status = core.add_light_directional(
    angularDiameter=0.5, color=[1,1,1], direction=[-0.7,-0.4,-1],
    intensity=1.0, is_visible=False
)